In [65]:
import pandas as pd
import sqlite3
import numpy as np

In [66]:
conn = sqlite3.connect('data2.db')

In [67]:
customers = pd.read_sql('select * from customer', conn)
transactions = pd.read_sql('select * from transactions', conn, parse_dates='transaction_dttm')

display(customers)
display(transactions)

#transactions.concat([10, 101, 600, 1, pd.to_datetime('2024-03-17 08:45:00')])
addit = pd.DataFrame([[10, 101, 600, 1, pd.to_datetime('2024-03-17 08:45:00')],
                      [11, 101, 700, 1, pd.to_datetime('2024-03-17 09:10:00')],
                      [12, 101, 650, 1, pd.to_datetime('2024-03-17 09:00:00')]],
                     columns=transactions.columns)

display(addit)

#transactions = pd.concat([transactions, addit])


,id,name,start_dttm,birth_dt
0,101,Иванов Иван,2024-01-15,1990-05-10
1,102,Петрова Мария,2024-02-20,1985-11-25
2,103,Сидоров Алексей,2024-03-05,1982-08-15
3,104,Козлова Екатерина,2024-01-02,1995-04-30
4,105,Смирнов Дмитрий,2024-03-10,1977-09-20


,id,customer_id,amount_rur,success_flg,transaction_dttm
0,1,101,500.00,1,2024-03-17 08:30:00
1,2,102,750.25,0,2024-03-17 09:45:00
2,3,103,1000.50,1,2024-03-17 10:20:00
3,4,101,300.75,1,2024-03-17 11:00:00
4,5,104,1200.80,0,2024-03-17 12:30:00
5,6,101,600.75,1,2024-04-17 11:00:00
6,7,101,200.75,1,2024-04-18 11:00:00
7,8,103,1000.50,1,2024-04-17 10:20:00
8,9,102,350.00,0,2024-04-17 09:45:00


,id,customer_id,amount_rur,success_flg,transaction_dttm
0,10,101,600,1,2024-03-17 08:45:00
1,11,101,700,1,2024-03-17 09:10:00
2,12,101,650,1,2024-03-17 09:00:00


In [68]:
def process(customers: pd.DataFrame, transactions: pd.DataFrame):

    #===========create datamart=================
    display('===========create datamart================')

    df = pd.merge(customers, transactions, left_on='id', right_on='customer_id')
    df = df[df['success_flg'] == True]
    df.sort_values(['name', 'transaction_dttm'], inplace=True)

    display(df)
    


    #============process payments================
    display('============process payments================')

    rur = df['amount_rur']
    display(rur)

    dif = np.diff(rur)
    display('dif is', dif)
    display('previous', dif[:-1])

    display('next', dif[1:])

    print('so result is', (dif[:-1] > 0) * (dif[1:] > 0))

    bool_diff = np.array([False, *((dif[:-1] > 0) * (dif[1:] > 0)), False])
    display(bool_diff)



    #=========process names====================
    display('=========process names====================')

    names = np.array(df['name'])

    bool_names = (names[:-2] == names[1:-1]) * (names[2:] == names[1:-1])
    display(bool_names)

    bool_names = np.array([False, *bool_names, False])
    display(bool_names)


    #apply masks to dataframe
    display('====================result=======================')
    return df[bool_diff & bool_names]


In [69]:
process(customers ,transactions)

'===========create datamart================'

,id_x,name,start_dttm,birth_dt,id_y,customer_id,amount_rur,success_flg,transaction_dttm
0,101,Иванов Иван,2024-01-15,1990-05-10,1,101,500.00,1,2024-03-17 08:30:00
1,101,Иванов Иван,2024-01-15,1990-05-10,4,101,300.75,1,2024-03-17 11:00:00
2,101,Иванов Иван,2024-01-15,1990-05-10,6,101,600.75,1,2024-04-17 11:00:00
3,101,Иванов Иван,2024-01-15,1990-05-10,7,101,200.75,1,2024-04-18 11:00:00
6,103,Сидоров Алексей,2024-03-05,1982-08-15,3,103,1000.50,1,2024-03-17 10:20:00
7,103,Сидоров Алексей,2024-03-05,1982-08-15,8,103,1000.50,1,2024-04-17 10:20:00


'============process payments================'

0     500.00
1     300.75
2     600.75
3     200.75
6    1000.50
7    1000.50
Name: amount_rur, dtype: float64

'dif is'

...

'=========process names===================='

array([ True,  True, False, False])

array([False,  True,  True, False, False, False])

'====================result======================='

,id_x,name,start_dttm,birth_dt,id_y,customer_id,amount_rur,success_flg,transaction_dttm
